# Assignment 1

## By Team 1     (Joey, Ryan, Vinit)    WES-237A.

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

### Use gpio.ipynb as starting point and add C++ function to reset all GPIO pins. PMODB selected 


### gpio.ipynb has issue. Consistingly opening a GPIO of a particular ID would create sub childs. So, after opening 15 instances, issue would come. Issue could be replicated if we turn LED on of a particular color 15 times. 
https://github.com/Xilinx/PYNQ/blob/master/boards/sw_repo/pynqmb/src/gpio.c
### I used c++ class object concept explained in the following link
https://github.com/Xilinx/PYNQ_Workshop/blob/master/Session_3/5_microblaze_programming.ipynb
### creating class-like objects

In [186]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"
#include "timer.h"
#include "stdint.h"

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

/** Clears the value of all the pins of GPIO of Microblaze PMODB
 *
 * Instances should be called using clear_gpio
 *
 */
void clear_gpio()
{
    for(int i=0; i<=7; i++)
    {
        write_gpio(i, 0);
    }
}

/** Class for interacting with Microblaze PMODB GPIO
 *
 * Instances should be created using `pmodb_gpio_open`
 *
 */
typedef unsigned int t1pmodb_gpio;

/** Returns object of Class for interacting with Microblaze PMODB GPIO
 *
 * Instances should be created using `pmodb_gpio_open`
 *
 */
t1pmodb_gpio t1pmodb_gpio_open(unsigned int pin, unsigned int direction)
{
    gpio pin_id = gpio_open(pin);
    gpio_set_direction(pin_id, direction);
    
    return pin_id;
}

/** Writes a value to object of GPIO pin of object of Class Microblaze PMODB
 *
 * Instances should be called using `obj.write(val)`
 *
 */
void t1pmodb_gpio_write(t1pmodb_gpio pin_id, unsigned int val)
{
    if (val > 1)
    {
        pyprintf("pin value must be 0 or 1");
    }
    gpio_write(pin_id, val);
}

/** Reads a value of object of GPIO pin of object of Class Microblaze PMODB
 *
 * Instances should be called using `obj.read()`
 *
 */
unsigned int t1pmodb_gpio_read(t1pmodb_gpio pin_id)
{
    return gpio_read(pin_id);
}

/** Clears the value of object of GPIO pin of object of Class Microblaze PMODB
 *
 * Instances should be called using `obj.clear()`
 *
 */
void t1pmodb_gpio_clear(t1pmodb_gpio pin_id)
{
    gpio_write(pin_id, 0);
}

### 1. Create a C++ function to clear all the GPIO pins of PMODB

#### Note, The C++ method used in the lab had issues when continuously writing on a GPIO for about 15 times. I implemented simple loop to just write on a GPIO and sleep for 1 sec and clear it and no other functionality. Found the LED remains on and no further changes to it. So I implemented Class-like implementation for GPIO. With that I am not seeing this issue.

In [85]:
clear_gpio()

### Observation: When GPIO is initialized using microblaze PMODB, The sensor has overlapping of green and red color.  On creating the object instance, the led is reset, ie no color is seen
### Creating object instances of GPIO pins and testing write and read

In [89]:
import time

GPIO_OUT = 0
GPIO_IN = 1
blue_led = t1pmodb_gpio_open(1,GPIO_OUT)
green_led = t1pmodb_gpio_open(2, GPIO_OUT)
red_led = t1pmodb_gpio_open(3, GPIO_OUT)
blue_led.write(1)
green_led.write(1)
red_led.write(1)
print("Blue Led = ",blue_led.read(), "Green Led = ",green_led.read(), "Red Led = ",red_led.read())
time.sleep(1)
blue_led.clear()
green_led.clear()
red_led.clear()
print("Blue Led = ",blue_led.read(), "Green Led = ",green_led.read(), "Red Led = ",red_led.read())

Blue Led =  1 Green Led =  1 Red Led =  1
Blue Led =  0 Green Led =  0 Red Led =  0


## Creating test function to consistently blinking LED

In [87]:
def test_PMODBLed():
    for x in range(50): 
        blue_led.write(1)
        time.sleep(1)
        blue_led.clear()
        green_led.write(1)
        time.sleep(1)
        green_led.clear()
        red_led.write(1)
        time.sleep(1)
        red_led.clear()

In [88]:
test_PMODBLed()

## Adding PWM Functionality 

In [187]:
#definition to turn all led pins off
def turn_leds_off():
    for color in pins:
        pins[color].write(0)
        
#definition to turn off any particular LED        
def turn_led_off(color):
    pins[color].write(0)
    
#definition to calculate cycles to be run freq in Hz, desired time in second
def calc_cycles(freq,desired_time):
    period = round(1.0/freq,6)
    cycles = round(desired_time/period)
    return cycles

# calculate time period of frequency in Hz, time period in sec
def calc_time_period(freq):
    return round(1.0/freq, 6)

# calculate ton based on duty cycle for a particular time period of a signal. Duty cycle in percentage
def calc_ton(duty_cycle, time_period):
    ton = round(0.01*duty_cycle*time_period,6)
    return ton

# PWM generator.freqHz in Hz, duty_cycle in percentage, color, run_time in second
def generate_pwm(freqHz, duty_cycle, color, run_time, supress_output=True):   
    if freqHz <= 0:
        print("Frequency should be greater than zero")
    if not((color == 'red') or (color == "blue") or (color == 'green')):
        print("Invalid input. Enter 'red', 'green or 'blue' as first arg.")
        return 
    if duty_cycle < 1:
        turn_led_off(color)
        print('dutycycle < 1, Led would be turned off. Please use Duty cycle >1')
        return 
    if duty_cycle > 99:
        pins[color].write(1)
        print('duty_cycle > 99, Please use Duty Cycle less than 100.')
        return 
    if run_time <= 0: #runtime could be half seconds
        print("run_time must be >= 0. preferrably 1 or more")
        return 
    
    time_period = calc_time_period(freqHz)
    ton = calc_ton(duty_cycle,time_period)
    toff = round(time_period - ton, 6)
    num_cycles = calc_cycles(freqHz, run_time)
    
    while num_cycles:
        pins[color].write(1)
        time.sleep(ton)
        pins[color].clear()
        time.sleep(toff)
        num_cycles -= 1
   
    if not(supress_output):
        print("Period:   {}".format(time_period))
        print("Time On:  {}".format(ton))
        print("Time Off: {}".format(toff))
        print("done!")

In [188]:
GPIO_OUT = 0
GPIO_IN = 1

#creat PMODB Gpio Objects

pin_1 = t1pmodb_gpio_open(1,GPIO_OUT)
pin_2 = t1pmodb_gpio_open(2, GPIO_OUT)
pin_3 = t1pmodb_gpio_open(3, GPIO_OUT)

#store these pins in a dictionary to easily refrence based on color
pins = {
    'red'   : pin_3, 
    'green' : pin_2,
    'blue'  : pin_1
}


In [111]:
pins['red'].write(1)
pins['green'].write(1)
pins['blue'].write(1)
time.sleep(1)
turn_leds_off()

In [112]:
pins['green'].write(1)
time.sleep(1)
turn_led_off('green')

### Emulate PWM for a given frequency and duty cycle

In [191]:
freqHz = 45
duty_cycle = 25
color = 'blue'
run_time = 1

def test_pwm():
    for x in range(15):
        generate_pwm(freqHz, duty_cycle, color, run_time)
        turn_led_off(color)
        time.sleep(1)


In [192]:
test_pwm()

### 5. Plot of perceived brightness vs duty cycle

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

xDutyCycle = np.array([1, 10, 20, 25, 30, 40, 50, 60, 70, 75, 80, 90, 99])
yBrightness = np.array([10, 25, 40, 50, 75, 85, 90, 90, 100, 100, 100, 100, 100])

plt.plot(xDutyCycle, yBrightness)
plt.title("Plot of LED brightness and duty cycle")
plt.xlabel("Duty Cycle in percentage")
plt.ylabel("Perceived brightness in percentage")
plt.show()

### 6 Async PWM

In [220]:
import time
from pynq.lib import Pmod_IO
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [221]:
#creat Pmos_IO objects for all used pins
gpio_pin_1 = Pmod_IO(base.PMODB,1,'out')
gpio_pin_2 = Pmod_IO(base.PMODB,2,'out')
gpio_pin_3 = Pmod_IO(base.PMODB,3,'out')

#store thses pins in a dictionary to easily refrence based on color
pins = {
    'red'   : gpio_pin_3, 
    'green' : gpio_pin_2,
    'blue'  : gpio_pin_1
} 

In [222]:
def turn_leds_off():
    for color in pins:
        pins[color].write(0)

In [216]:
pins['blue'].write(1) #turns on blue led

In [223]:
turn_leds_off()

In [ ]:
import asyncio

#power states
ON = 1
OFF = 0

color = 'red'

cond = True
state = ON

btns = base.btns_gpio

freq = 50 #Hz
duty_cycle = 25 #percent
period = 1/freq
t_on = period*duty_cycle/100
t_off = period - t_on
time_pwm_on = 1
time_pwm_off = 1
numIter = round(time_pwm_on/period)
async def flash_leds():
    global cond, state
    while cond:
        if state == ON:
            for i in range(numIter):
                if state == OFF:
                    break
                pins[color].write(1)
                await asyncio.sleep(t_on)
                turn_leds_off()
                await asyncio.sleep(t_off)
            pins[color].write(0)    
            await asyncio.sleep(time_pwm_off) #off for 1 sec
        else:
            turn_leds_off()
            await asyncio.sleep(0.05)

async def get_btns(_loop):
    global cond, state, color
    while cond:
        await asyncio.sleep(0.01)
        btn = btns.read()
        if btn == 0x01:
            color = 'red'
            state = ON
            print("red button pressed")
        elif btn == 0x02:
            color = 'green'
            state = ON
            print("green button pressed")
        elif btn == 0x04:
            color = 'blue'
            state = ON
            print("blue button pressed")
        elif btn == 0x08:
            turn_leds_off()
            state = OFF
            print("off button pressed")
        elif btn > 0x08:
            #press the off button and another at the same time to terminate loop
            turn_leds_off()
            state = OFF
            print("Loop Terminated.")
            _loop.stop()

loop = asyncio.new_event_loop()
loop.create_task(flash_leds())
loop.create_task(get_btns(loop))
loop.run_forever()
loop.close()        
print("1st assignment Done.")